In [64]:
from pystats_utils.utils.patient import Patient

import pandas as pd

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
for index, row in data.iterrows():
    patients.append(Patient(row))

In [65]:
for patient in patients:
    treatment = patient.translateTreatmentLine(only = "all",
                                               condensed = False)

    test = [i for j in treatment for i in j]

    if "Caz-avi" in test: print(patient.pt, treatment)

46 [{'Inactive'}, {'Caz-avi', 'Inactive'}, {'Caz-avi', 'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}, {'Meropenem'}]
116 [{'Ceftriaxone'}, {'Ceftriaxone', 'Inactive'}, {'Inactive'}, {'Caz-avi', 'Inactive'}, {'Ceftriaxone', 'Caz-avi'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone'}, {'Ceftriaxone', 'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}, {'Cefixime'}]


In [74]:
import pandas as pd
from pystats_utils.utils.patient import Patient
from pystats_utils.utils.treatment_plot import generatePlot

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
#data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "yes"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "Caz-avi Cohort")

a.save('treatment_cazavi.png', dpi=300)

data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
#data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "BAT Cohort")
a.save('treatment_bat.png', dpi=300)


data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "yes"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "Caz-avi Cohort alive")
a.save('treatment_cazavi_alive.png', dpi=300)


data = pd.read_csv("./test/database/treatment.csv",
                   sep = "\t")
data = data[data["Cohort_Treatment"] == "yes"]
data = data[data["Mortality_31d"] == "no"]
data = data[data["Cohort_Cazavi"] == "no"]

patients = []
for index, row in data.iterrows():

    patient = Patient(row)

    patients.append(patient)

a = generatePlot(patients, days = 30, title = "BAT Cohort alive")
a.save('treatment_bat_alive.png', dpi=300)


/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: treatment_cazavi.png
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: treatment_bat.png
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: Filename: treatment_cazavi_alive.png
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:719: PlotnineWarning: Saving 20 x 20 in image.
/home/virtualvikings/anaconda3/lib/python3.9/site-packages/plotnine/ggplot.py:722: PlotnineWarning: File